# Python调用Gurobi求解Multicommodity Network Flow Problem 

In [1]:
from gurobipy import *
import pandas as pd 
import numpy as np
from pandas import Series, DataFrame
import math 

In [2]:
Arcs = {'1,2': [15, 15]    # cost flow 
        ,'1,4': [25, 25]
        ,'1,3': [45, 45]
        ,'2,5': [30, 60]
        ,'2,4': [2, 2]
        ,'5,7': [2, 2]
        ,'4,7': [50, 100]
        ,'4,3': [2, 2]
        ,'3,6': [25, 50]
        ,'6,7': [1, 1]
       }
Arcs 

{'1,2': [15, 15],
 '1,4': [25, 25],
 '1,3': [45, 45],
 '2,5': [30, 60],
 '2,4': [2, 2],
 '5,7': [2, 2],
 '4,7': [50, 100],
 '4,3': [2, 2],
 '3,6': [25, 50],
 '6,7': [1, 1]}

In [3]:
Nodes = [1, 2, 3, 4, 5, 6, 7] 

commodity = [[1, 7, 25],  # s_i, d_i, demand 
             [2, 6, 2]]

In [4]:
model = Model('MultiCommodity')  

Using license file E:\Anaconda3\win64\gurobi.lic


In [5]:
# add variables 
X = {}
for key in Arcs.keys():
    for k in range(len(commodity)):
        key_x = key + ',' + str(k)
        X[key_x] = model.addVar(lb=0
                                ,ub=Arcs[key][1]
                                ,vtype=GRB.CONTINUOUS
                                ,name= 'x_' + key_x 
                               ) 

In [6]:
# add objective function 
obj = LinExpr(0)
for key in Arcs.keys():
    for k in range(len(commodity)):
        key_x = key + ',' + str(k)
        obj.addTerms(Arcs[key][0], X[key_x])
model.setObjective(obj, GRB.MINIMIZE)

In [7]:
# constraints 1 
for k in range(len(commodity)):
    for i in Nodes:
        lhs = LinExpr(0)
        for key_x in X.keys():
#             nodes = key_x.split(',')
            if(i == (int)(key_x.split(',')[0]) and k == (int)(key_x.split(',')[2])):
                lhs.addTerms(1, X[key_x])
            if(i == (int)(key_x.split(',')[1]) and k == (int)(key_x.split(',')[2])):
                lhs.addTerms(-1, X[key_x])
        if(i == commodity[k][0]):
            model.addConstr(lhs == commodity[k][2], name='org_, ' + str(i) + '_' + str(k))
        elif(i == commodity[k][1]): 
            model.addConstr(lhs == -commodity[k][2], name='des_, ' + str(i) + '_' + str(k))
        else:
            model.addConstr(lhs == 0, name='inter_, ' + str(i) + '_' + str(k))
            

In [8]:
# constraints 2  
for key in Arcs.keys():
    lhs = LinExpr(0)
    for k in range(len(commodity)):
        key_x = key + ',' + str(k)
        lhs.addTerms(1, X[key_x])
    model.addConstr(lhs <= Arcs[key][1], name = 'capacity_, ' + key) 

In [9]:
model.write('Multicommodity_model.lp')
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 24 rows, 20 columns and 60 nonzeros
Model fingerprint: 0xcc40d44b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]
Presolve removed 24 rows and 20 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8730000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.08 seconds
Optimal objective  1.873000000e+03


In [10]:
for var in model.getVars():
    if(var.x > 0):
        print(var.varName, '\t', var.x) 

x_1,2,0 	 2.0
x_1,4,0 	 22.0
x_1,3,0 	 1.0
x_2,5,0 	 2.0
x_2,4,1 	 2.0
x_5,7,0 	 2.0
x_4,7,0 	 22.0
x_4,3,1 	 2.0
x_3,6,0 	 1.0
x_3,6,1 	 2.0
x_6,7,0 	 1.0


In [11]:
dual = model.getAttr("Pi", model.getConstrs())

In [12]:
dual 

[75.0,
 60.0,
 30.0,
 50.0,
 30.0,
 5.0,
 0.0,
 0.0,
 10.0,
 -20.0,
 0.0,
 0.0,
 -45.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -8.0,
 0.0,
 0.0,
 -18.0,
 0.0,
 0.0]